In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Input, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l2

# Download data for Tata Motors (TATAMOTORS.NS)
ticker = 'TATAMOTORS.NS'
data = yf.download(ticker, start="2000-01-01", end="2024-01-01")

# Feature selection ('Open', 'High', 'Low', 'Close', 'Volume')
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]

# Data preprocessing: Scale the features using MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Create sequences for the models
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length, 3])  # Predicting 'Close' price
    return np.array(X), np.array(y)

seq_length = 30  # Using 30 days of data to predict the next day's price
X, y = create_sequences(scaled_data, seq_length)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape X for Conv1D and LSTM
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

# 1. CNN Model
cnn_input = Input(shape=(seq_length, X.shape[2]))
cnn_layer = Conv1D(64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01))(cnn_input)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)
cnn_layer = Dropout(0.5)(cnn_layer)
cnn_layer = Flatten()(cnn_layer)
cnn_layer = Dense(64, activation='relu')(cnn_layer)
cnn_output = Dense(1)(cnn_layer)

cnn_model = Model(inputs=cnn_input, outputs=cnn_output)
cnn_model.compile(optimizer='adam', loss='mse')

# 2. LSTM Model
lstm_input = Input(shape=(seq_length, X.shape[2]))
lstm_layer = LSTM(64, return_sequences=False, kernel_regularizer=l2(0.01))(lstm_input)
lstm_layer = Dropout(0.5)(lstm_layer)
lstm_layer = Dense(64, activation='relu')(lstm_layer)
lstm_output = Dense(1)(lstm_layer)

lstm_model = Model(inputs=lstm_input, outputs=lstm_output)
lstm_model.compile(optimizer='adam', loss='mse')

# Early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Train the CNN model
history_cnn = cnn_model.fit(X_train_cnn, y_train, epochs=200, batch_size=64, validation_split=0.2, 
                            callbacks=[early_stopping, reduce_lr])

# Train the LSTM model
history_lstm = lstm_model.fit(X_train_cnn, y_train, epochs=200, batch_size=64, validation_split=0.2, 
                              callbacks=[early_stopping, reduce_lr])

# 3. Combine CNN and LSTM outputs for ensemble prediction
# Get predictions from both models
cnn_predictions = cnn_model.predict(X_test_cnn)
lstm_predictions = lstm_model.predict(X_test_cnn)

# Simple averaging of predictions from CNN and LSTM
ensemble_predictions = (cnn_predictions + lstm_predictions) / 2

# Plot the results
plt.figure(figsize=(14,5))
plt.plot(y_test, color='blue', label='Actual Stock Price')
plt.plot(ensemble_predictions, color='red', label='Ensemble Predicted Stock Price')
plt.title(f'{ticker} Stock Price Prediction (CNN + LSTM Ensemble)')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

# Evaluate the ensemble model
ensemble_loss = np.mean((ensemble_predictions - y_test) ** 2)
print(f'Ensemble Model Test Loss: {ensemble_loss}')
